In [ ]:
#Fixed to use altitude/GSD from each individual image to calculate ground distance from drone of individual annotations

import pandas as pd
import ast

# Read the two CSV files
df_annotations = pd.read_csv("C:\\...\site_anns.csv") #COCO format SAHI outputs
df_annotations['bbox'] = df_annotations['bbox'].apply(ast.literal_eval)
df_resolution = pd.read_csv("C:\\...\\site_metadata.csv") # "derive_agl_gsd.ipynb" output

# Merge the two dataframes based on the 'filename' column
merged_df = df_annotations.merge(df_resolution, on='filename', how='inner')

# Image center in pixels
center_x_px, center_y_px = 2736, 1824  # Specify according to your image set

# Calculate the center point in meters
merged_df['center_x_m'] = center_x_px * merged_df['gsd']
merged_df['center_y_m'] = center_y_px * merged_df['gsd']

# Function to calculate distance from center
def calculate_distance_from_center(row):
    # Get the coordinates of the bounding box (x, y, width, height)
    x, y, width, height = row['bbox']

    # Calculate the center point of the bounding box in pixels
    bbox_center_x_px = x + (width / 2)
    bbox_center_y_px = y + (height / 2)

    # Calculate the center point of the bounding box in meters
    bbox_center_x_m = bbox_center_x_px * row['gsd']
    bbox_center_y_m = bbox_center_y_px * row['gsd']

    # Calculate the distance from the center of the image in meters
    distance_m = ((row['center_x_m'] - bbox_center_x_m)**2 + (row['center_y_m'] - bbox_center_y_m)**2)**0.5

    return distance_m

# Apply the function to the merged dataframe
merged_df['distance_from_center'] = merged_df.apply(calculate_distance_from_center, axis=1)

# Export the modified dataframe to CSV
savepath = "C:\\...\\"  # Desired output directory
merged_df.to_csv(savepath + "distance_anns.csv", index=False)  # Desired filename
